In [20]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn
import math
%matplotlib inline
import os

import sklearn
import scipy.io as sio
from numba import jit
from math import factorial, log
from sklearn.neighbors import KDTree
from scipy.signal import periodogram, welch
from astropy.timeseries import LombScargle
import re

In [21]:
'''
                                                                 
for j in range(1):                                                                  #action
    path = os.path.abspath(os.getcwd())+'\\sensor_csv'+'\\'+str(i)+'\\a'+str(j+1)
    file = os.listdir(path)
    file1 = str(file)
    print(file1)  
    file1 = file1.replace("[", "")
    file1 = file1.replace("]", "")
    file1 = file1.replace("'", "")
    print(file1) 
    filelist = file1.split(",")
    
    

#print(files)
'''

'\n                                                                 \nfor j in range(1):                                                                  #action\n    path = os.path.abspath(os.getcwd())+\'\\sensor_csv\'+\'\\\'+str(i)+\'\\a\'+str(j+1)\n    file = os.listdir(path)\n    file1 = str(file)\n    print(file1)  \n    file1 = file1.replace("[", "")\n    file1 = file1.replace("]", "")\n    file1 = file1.replace("\'", "")\n    print(file1) \n    filelist = file1.split(",")\n    \n    \n\n#print(files)\n'

In [22]:
mat_filenames = []
filenames = []
for file1 in filelist:
    x = re.split("_|\.",file1)
    #print(x)
    name = x[0]
    time = x[1]
    #print(type(file))
    for j in range(22):
        str1 = name+"_a"+str(j+1)+"_"+time+".mat"
        filename = name+"_a"+str(j+1)+"_"+time
        mat_filenames.append(str1)
        filenames.append(filename)


NameError: name 'filelist' is not defined

In [23]:
path = os.path.abspath(os.getcwd())+'\\sensor_csv'+'\\1'+'\\a4'
files = os.listdir(path)
file = files[0]
df = pd.read_csv(path+'//'+file)

In [24]:
print(df['a_x'].shape)
print(df['ts'].shape)

(4216,)
(4216,)


In [25]:
#print(files)
print(len(filelist))
print(len(mat_filenames))
print(len(filenames))

NameError: name 'filelist' is not defined

In [26]:
def spectral_entropy(x, sf=100, method='fft', nperseg=None, normalize=False,axis=-1):
    """Spectral Entropy.
    Parameters
    ----------
    x : list or np.array
        1D or N-D data.
    sf : float
        Sampling frequency, in Hz.
    method : str
        Spectral estimation method:
        * ``'fft'`` : Fourier Transform (:py:func:`scipy.signal.periodogram`)
        * ``'welch'`` : Welch periodogram (:py:func:`scipy.signal.welch`)
    nperseg : int or None
        Length of each FFT segment for Welch method.
        If None (default), uses scipy default of 256 samples.
    normalize : bool
        If True, divide by log2(psd.size) to normalize the spectral entropy
        between 0 and 1. Otherwise, return the spectral entropy in bit.
    axis : int
        The axis along which the entropy is calculated. Default is -1 (last).
    Returns
    -------
    se : float
        Spectral Entropy
    Notes
    -----
    Spectral Entropy is defined to be the Shannon entropy of the power
    spectral density (PSD) of the data:
    .. math:: H(x, sf) =  -\\sum_{f=0}^{f_s/2} P(f) \\log_2[P(f)]
    Where :math:`P` is the normalised PSD, and :math:`f_s` is the sampling
    frequency.
    References
    ----------
    - Inouye, T. et al. (1991). Quantification of EEG irregularity by
      use of the entropy of the power spectrum. Electroencephalography
      and clinical neurophysiology, 79(3), 204-210.
    - https://en.wikipedia.org/wiki/Spectral_density
    - https://en.wikipedia.org/wiki/Welch%27s_method
    Examples
    --------
    Spectral entropy of a pure sine using FFT
    >>> import numpy as np
    >>> import entropy as ent
    >>> sf, f, dur = 100, 1, 4
    >>> N = sf * dur # Total number of discrete samples
    >>> t = np.arange(N) / sf # Time vector
    >>> x = np.sin(2 * np.pi * f * t)
    >>> np.round(ent.spectral_entropy(x, sf, method='fft'), 2)
    0.0
    Spectral entropy of a random signal using Welch's method
    >>> np.random.seed(42)
    >>> x = np.random.rand(3000)
    >>> ent.spectral_entropy(x, sf=100, method='welch')
    6.980045662371389
    Normalized spectral entropy
    >>> ent.spectral_entropy(x, sf=100, method='welch', normalize=True)
    0.9955526198316071
    Normalized spectral entropy of 2D data
    >>> np.random.seed(42)
    >>> x = np.random.normal(size=(4, 3000))
    >>> np.round(ent.spectral_entropy(x, sf=100, normalize=True), 4)
    array([0.9464, 0.9428, 0.9431, 0.9417])
    Fractional Gaussian noise with H = 0.5
    >>> import stochastic.processes.noise as sn
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.5, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9505
    Fractional Gaussian noise with H = 0.9
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.9, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.8477
    Fractional Gaussian noise with H = 0.1
    >>> rng = np.random.default_rng(seed=42)
    >>> x = sn.FractionalGaussianNoise(hurst=0.1, rng=rng).sample(10000)
    >>> print(f"{ent.spectral_entropy(x, sf=100, normalize=True):.4f}")
    0.9248
    """
    x = np.asarray(x)
    #sf = np.array(sf, dtype=np.float64)
    # Compute and normalize power spectrum
    if method == 'fft':
        _, psd = periodogram(x, sf, axis=axis)
    elif method == 'welch':
        _, psd = welch(x, sf, nperseg=nperseg, axis=axis)
    psd_norm = psd / psd.sum(axis=axis, keepdims=True)
    se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
    if normalize:
        se /= np.log2(psd_norm.shape[axis])
    return se

In [27]:
def statf(c):
    col = 'a_'+c
    col1 = 'g_'+c
    # acc
    mean_a = df[col].mean()
    var_a = df[col].var()
    std_a = df[col].std()
    mx_a = df[col].max()
    mn_a = df[col].min()
    rng_a = mx_a-mn_a
    #mode = df[col].mode()
    #n_z = ((df[col][:-1] * df[col][1:]) < 0).sum()
    
    #gyro
    mean_g = df[col1].mean()
    var_g = df[col1].var()
    std_g = df[col1].std()
    mx_g = df[col1].max()
    mn_g = df[col1].min()
    rng_g = mx_g-mn_g
    
    f_a=abs(np.fft.fft(df[col]))
    f_g=abs(np.fft.fft(df[col1]))
    num=df.shape[0]
    
    #fs=1./diff(t)
    
    freq = [i / num for i in list(range(num))]
    
    spectrum_a = f_a.real*f_a.real+f_a.imag*f_a.imag
    dc_comp_a = spectrum_a[0]
    spectrum_a = np.multiply(spectrum_a,spectrum_a)  # get the sectrum square
    spec_energy_a = spectrum_a.sum()/num
    
    spectrum_g = f_g.real*f_g.real+f_g.imag*f_g.imag
    dc_comp_g = spectrum_g[0]
    spectrum_g = np.multiply(spectrum_g,spectrum_g)  # get the sectrum square
    spec_energy_g = spectrum_g.sum()/num
    
    # spectral entropy
    
#     df['ts_shifted'] = df['ts'].shift(1)
#     freq1=df['ts'] - df['ts_shifted']
#     freq1 = freq1.dropna(axis = 0, how = 'all')
    tim = df['ts'][df.shape[0]-1] - df['ts'][0]
    freq1 = df.shape[0]/tim
    mean_freq = np.mean(freq1)
    #mean_freq = 1/np.mean(freq1)
    spec_entropy_a = spectral_entropy(df[col], mean_freq, method='fft', nperseg=None, normalize=False,axis=-1)
    spec_entropy_g = spectral_entropy(df[col1], mean_freq, method='fft', nperseg=None, normalize=False,axis=-1)
    
    # poer spectral density using LombScargle Periodogram
    
    frequency_a, power_a = LombScargle(df['ts'], df[col]).autopower()

    max_psd_a = np.max(power_a)
    min_psd_a = np.min(power_a)
    min_max_psd_a = min_psd_a/max_psd_a
    
    max_xas_a = np.sqrt(max_psd_a)
    min_xas_a = np.sqrt(min_psd_a)
    min_max_xas_a = min_xas_a/max_xas_a
    
    frequency_g, power_g = LombScargle(df['ts'], df[col1]).autopower()

    max_psd_g = np.max(power_g)
    min_psd_g = np.min(power_g)
    min_max_psd_g = min_psd_g/max_psd_g
    
    max_xas_g = np.sqrt(max_psd_g)
    min_xas_g = np.sqrt(min_psd_g)
    min_max_xas_g = min_xas_g/max_xas_g

    return np.array([mean_a ,var_a ,std_a ,mx_a ,mn_a ,rng_a ,dc_comp_a ,spec_energy_a ,spec_entropy_a , 
                     max_psd_a ,min_psd_a ,min_max_psd_a ,max_xas_a ,min_xas_a ,min_max_xas_a ,
                    mean_g ,var_g ,std_g ,mx_g ,mn_g ,rng_g ,dc_comp_g ,spec_energy_g ,spec_entropy_g , 
                     max_psd_g ,min_psd_g ,min_max_psd_g ,max_xas_g ,min_xas_g ,min_max_xas_g ])
    

In [29]:
data = np.array([])
sensor0 = np.array([])
sensor1 = np.array([])
sensor2 = np.array([])
sensor3 = np.array([])
sensor4 = np.array([])
sensor5 = np.array([])
sensor6 = np.array([])
sensor7 = np.array([])
sensor8 = np.array([])
sensor9 = np.array([])

k = 0

# for each sensor position
for i in range(10):
    
    sensor = np.array([])
    k = 0
    #for each activity
    for j in range(22):
        
        path = os.path.abspath(os.getcwd())+'\\sensor_csv'+'\\'+str(i)+'\\a'+str(j+1)
        files = os.listdir(path)
        
        # for each time seq
        for file in files:
            
            # namemaking
            #print(file)
            x = re.split("_|\.",file)
            name = x[0]
            time = x[1]
            mat_filename = name+"_a"+str(j+1)+"_"+time+".mat"
            filename = name+"_a"+str(j+1)+"_"+time
            
            df = pd.read_csv(path+'//'+file)
            x = statf('x')
            y = statf('y')
            z = statf('z')
            df['a_m'] =  np.sqrt(df['a_x']*df['a_x']+df['a_y']*df['a_y']+df['a_z']*df['a_z'])
            df['g_m'] =  np.sqrt(df['g_x']*df['g_x']+df['g_y']*df['g_y']+df['g_z']*df['g_z'])
            m = statf('m')
            act = j+1
            print(filename+"---------------"+str(i))
            
            l = [act,mat_filename,filename]
            sen = np.append(np.concatenate((x, y, z, m), axis = 0),l)                  
            # add all the features together to make a row
            
            #dat = np.append(np.concatenate((x, y, z, m), axis = 0),act)
            #dat = np.append(dat,i)
            
            # add all the new rows
            if(k==0):
                sensor = sen
            else: 
                sensor =  np.vstack([sensor,sen])
            k+=1
            #print(k)
    if(i==0):
        sensor0 = sensor
        #print(len(sensor0))
    elif(i==1):
        sensor1 = sensor
    elif(i==2):
        sensor2 = sensor
    elif(i==3):
        sensor3 = sensor
    elif(i==4):
        sensor4 = sensor
    elif(i==5):
        sensor5 = sensor
    elif(i==6):
        sensor6 = sensor
    elif(i==7):
        sensor7 = sensor
    elif(i==8):
        sensor8 = sensor
    elif(i==9):
        sensor9 = sensor

cx_a1_t1---------------0
cx_a1_t10---------------0
cx_a1_t11---------------0
cx_a1_t12---------------0
cx_a1_t13---------------0
cx_a1_t2---------------0
cx_a1_t3---------------0
cx_a1_t4---------------0
cx_a1_t5---------------0
cx_a1_t6---------------0
cx_a1_t7---------------0
cx_a1_t8---------------0
cx_a1_t9---------------0
cyy_a1_t1---------------0
cyy_a1_t10---------------0
cyy_a1_t2---------------0
cyy_a1_t3---------------0
cyy_a1_t4---------------0
cyy_a1_t5---------------0
cyy_a1_t6---------------0
cyy_a1_t7---------------0
cyy_a1_t8---------------0
cyy_a1_t9---------------0
myj_a1_t1---------------0
myj_a1_t10---------------0
myj_a1_t2---------------0
myj_a1_t3---------------0
myj_a1_t4---------------0
myj_a1_t5---------------0
myj_a1_t6---------------0
myj_a1_t7---------------0
myj_a1_t8---------------0
myj_a1_t9---------------0
qyh_a1_t1---------------0
qyh_a1_t10---------------0
qyh_a1_t11---------------0
qyh_a1_t2---------------0
qyh_a1_t3---------------0
qyh_a1_t4--------

<ipython-input-26-b64774160073>:88: RuntimeWarning: divide by zero encountered in log2
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)
<ipython-input-26-b64774160073>:88: RuntimeWarning: invalid value encountered in multiply
  se = -(psd_norm * np.log2(psd_norm)).sum(axis=axis)


qyh_a2_t1---------------0
qyh_a2_t10---------------0
qyh_a2_t11---------------0
qyh_a2_t2---------------0
qyh_a2_t3---------------0
qyh_a2_t4---------------0
qyh_a2_t5---------------0
qyh_a2_t6---------------0
qyh_a2_t7---------------0
qyh_a2_t8---------------0
qyh_a2_t9---------------0
zyh_a2_t1---------------0
zyh_a2_t2---------------0
zyh_a2_t3---------------0
zyh_a2_t4---------------0
zyh_a2_t5---------------0
zyh_a2_t6---------------0
zyh_a2_t7---------------0
zyh_a2_t8---------------0
zyh_a2_t9---------------0
cx_a3_t1---------------0
cx_a3_t10---------------0
cx_a3_t11---------------0
cx_a3_t12---------------0
cx_a3_t13---------------0
cx_a3_t2---------------0
cx_a3_t3---------------0
cx_a3_t4---------------0
cx_a3_t5---------------0
cx_a3_t6---------------0
cx_a3_t7---------------0
cx_a3_t8---------------0
cx_a3_t9---------------0
cyy_a3_t1---------------0
cyy_a3_t10---------------0
cyy_a3_t2---------------0
cyy_a3_t3---------------0
cyy_a3_t4---------------0
cyy_a3_t5---------

In [30]:
print(len(sensor0))
print(len(sensor1))
print(len(sensor2))
print(len(sensor3))
print(len(sensor4))
print(len(sensor5))
print(len(sensor6))
print(len(sensor7))
print(len(sensor7))
print(len(sensor8))

1165
1165
1165
1165
1165
1165
1165
1165
1165
1165


In [31]:
sensor = sensor0

sensor = np.concatenate([sensor,sensor1],axis=1)
sensor = np.concatenate([sensor,sensor2],axis=1)
sensor = np.concatenate([sensor,sensor3],axis=1)
sensor = np.concatenate([sensor,sensor4],axis=1)
sensor = np.concatenate([sensor,sensor5],axis=1)
sensor = np.concatenate([sensor,sensor6],axis=1)
sensor = np.concatenate([sensor,sensor7],axis=1)
sensor = np.concatenate([sensor,sensor8],axis=1)
sensor = np.concatenate([sensor,sensor9],axis=1)

In [32]:
sensor.shape

(1165, 1230)

In [33]:

# Python program to iterate
# over 3 lists using zip function
 
import itertools

In [34]:
sen  = ['0_','1_','2_','3_','4_','5_','6_','7_','8_','9_']

# mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas

col = ['mean','var','std','max','min','range','dc_comp','spec_energy','spec_entropy','max_psd','min_psd','min_max_psd','max_xas','min_xas','min_max_xas']
axisa = ['_a_x','_g_x','_a_y','_g_y','_a_z','_g_z','_a_m','_g_m']
col1 = ['activity','mat_filename','filename']

col_name = []

for s in sen:
    for x in axisa:
        for (c) in (col):
            col_name.append(s+c+x)
            
#     for (c) in (col):
#             col_name.append(s+c+x)
            
    for c1 in col1:
        col_name.append(s+c1)

    
# col_name.append('activity')
# col_name.append('position')
print(col_name)

['0_mean_a_x', '0_var_a_x', '0_std_a_x', '0_max_a_x', '0_min_a_x', '0_range_a_x', '0_dc_comp_a_x', '0_spec_energy_a_x', '0_spec_entropy_a_x', '0_max_psd_a_x', '0_min_psd_a_x', '0_min_max_psd_a_x', '0_max_xas_a_x', '0_min_xas_a_x', '0_min_max_xas_a_x', '0_mean_g_x', '0_var_g_x', '0_std_g_x', '0_max_g_x', '0_min_g_x', '0_range_g_x', '0_dc_comp_g_x', '0_spec_energy_g_x', '0_spec_entropy_g_x', '0_max_psd_g_x', '0_min_psd_g_x', '0_min_max_psd_g_x', '0_max_xas_g_x', '0_min_xas_g_x', '0_min_max_xas_g_x', '0_mean_a_y', '0_var_a_y', '0_std_a_y', '0_max_a_y', '0_min_a_y', '0_range_a_y', '0_dc_comp_a_y', '0_spec_energy_a_y', '0_spec_entropy_a_y', '0_max_psd_a_y', '0_min_psd_a_y', '0_min_max_psd_a_y', '0_max_xas_a_y', '0_min_xas_a_y', '0_min_max_xas_a_y', '0_mean_g_y', '0_var_g_y', '0_std_g_y', '0_max_g_y', '0_min_g_y', '0_range_g_y', '0_dc_comp_g_y', '0_spec_energy_g_y', '0_spec_entropy_g_y', '0_max_psd_g_y', '0_min_psd_g_y', '0_min_max_psd_g_y', '0_max_xas_g_y', '0_min_xas_g_y', '0_min_max_xas_g

In [35]:
len(col_name)

1230

In [36]:
xD = pd.DataFrame(sensor)
xD.columns = col_name

In [38]:
xD.to_csv('sensor_data.csv', index=False)

In [39]:
xD.head()

,0_mean_a_x,0_var_a_x,0_std_a_x,0_max_a_x,0_min_a_x,0_range_a_x,0_dc_comp_a_x,0_spec_energy_a_x,0_spec_entropy_a_x,0_max_psd_a_x,...,9_spec_entropy_g_m,9_max_psd_g_m,9_min_psd_g_m,9_min_max_psd_g_m,9_max_xas_g_m,9_min_xas_g_m,9_min_max_xas_g_m,9_activity,9_mat_filename,9_filename
0,0.08440677527151225,0.005099253183426285,0.07140905533212356,0.24560500000000002,-0.108154,0.35375900000000005,40832.2121548324,738671.4967301019,3.2067189675424275,0.4728212690903644,...,5.0868878745765835,0.1942346041985453,4.0039035887254843e-10,2.0613750084576697e-09,0.44072055114158826,2.0009756592036507e-05,4.54023678728067e-05,1,cx_a1_t1.mat,cx_a1_t1
1,0.05868655153061222,0.00557284670697336,0.07465150170608331,0.238037,-0.14868199999999998,0.386719,13230.898087460882,132037.6017507014,2.8378307843951722,0.6569401400238601,...,5.135781149742887,0.21621238847078444,5.541471171876607e-10,2.5629757901802164e-09,0.46498643901815506,2.354032958961409e-05,5.0625841130594724e-05,1,cx_a1_t10.mat,cx_a1_t10
2,0.09736474987133306,0.005283285950662429,0.07268621568538583,0.280518,-0.11377000000000001,0.39428799999999997,35788.962297324695,701134.9325655709,2.7171058541770092,0.6296151794730955,...,5.560043683051307,0.13735482472074056,2.501386569172681e-09,1.8211130000408146e-08,0.3706141183505299,5.001386376968571e-05,0.00013494861985366188,1,cx_a1_t11.mat,cx_a1_t11
3,0.06759654350927238,0.0037317322946421347,0.06108790628792359,0.17871099999999998,-0.13623,0.31494099999999997,20208.19499389196,211783.71469614343,3.3576168310313275,0.5779111521277593,...,5.161082269776515,0.11146937379258033,9.195829665884523e-11,8.249646833932981e-10,0.33387029486400904,9.589488863273434e-06,2.872219844289949e-05,1,cx_a1_t12.mat,cx_a1_t12
4,0.07944031100000015,0.005409067411984266,0.07354636233005862,0.222412,-0.09887699999999999,0.321289,25243.05204710689,374472.95690838876,2.214226934810065,0.7474342064044721,...,5.2392551699103675,0.23452612404817988,4.7834495535008886e-11,2.0396233353168795e-10,0.4842789733698748,6.916248660582476e-06,1.4281538206078782e-05,1,cx_a1_t13.mat,cx_a1_t13


In [44]:
act = xD['0_activity']
mfn = xD['0_mat_filename']
fn = xD['0_filename']

d_col = []
for s in sen:
    for c1 in col1:
        d_col.append(s+c1)

xD.drop(d_col, axis = 1, inplace = True)
xD['activity'] = act
xD['mat_filename'] = mfn
xD['filename'] = fn

In [45]:
xD = xD.fillna(xD.mean())
is_NaN = xD.isnull()
row_has_NaN = is_NaN.any(axis=1)
xD[row_has_NaN]

,0_mean_a_x,0_var_a_x,0_std_a_x,0_max_a_x,0_min_a_x,0_range_a_x,0_dc_comp_a_x,0_spec_energy_a_x,0_spec_entropy_a_x,0_max_psd_a_x,...,9_spec_entropy_g_m,9_max_psd_g_m,9_min_psd_g_m,9_min_max_psd_g_m,9_max_xas_g_m,9_min_xas_g_m,9_min_max_xas_g_m,activity,mat_filename,0_filename


In [46]:
# xD.drop(['0_filename'], axis = 1, inplace = True)
# xD['filename'] = fn

In [48]:
sen  = ['0_','1_','2_','3_','4_','5_','6_','7_','8_','9_']

# mean,var,std,mx,mn,rng,dc_comp,spec_energy,spec_entropy, max_psd,min_psd,min_max_psd,max_xas,min_xas,min_max_xas

col = ['mean','var','std','max','min','range','dc_comp','spec_energy','spec_entropy','max_psd','min_psd','min_max_psd','max_xas','min_xas','min_max_xas']
axisa = ['_a_x','_g_x','_a_y','_g_y','_a_z','_g_z','_a_m','_g_m']
col1 = ['activity','mat_filename','filename']

col_name = []

for s in sen:
    for x in axisa:
        for (c) in (col):
            col_name.append(s+c+x)
for c1 in col1:
    col_name.append(c1)

    
# col_name.append('activity')
# col_name.append('position')
print(col_name)

['0_mean_a_x', '0_var_a_x', '0_std_a_x', '0_max_a_x', '0_min_a_x', '0_range_a_x', '0_dc_comp_a_x', '0_spec_energy_a_x', '0_spec_entropy_a_x', '0_max_psd_a_x', '0_min_psd_a_x', '0_min_max_psd_a_x', '0_max_xas_a_x', '0_min_xas_a_x', '0_min_max_xas_a_x', '0_mean_g_x', '0_var_g_x', '0_std_g_x', '0_max_g_x', '0_min_g_x', '0_range_g_x', '0_dc_comp_g_x', '0_spec_energy_g_x', '0_spec_entropy_g_x', '0_max_psd_g_x', '0_min_psd_g_x', '0_min_max_psd_g_x', '0_max_xas_g_x', '0_min_xas_g_x', '0_min_max_xas_g_x', '0_mean_a_y', '0_var_a_y', '0_std_a_y', '0_max_a_y', '0_min_a_y', '0_range_a_y', '0_dc_comp_a_y', '0_spec_energy_a_y', '0_spec_entropy_a_y', '0_max_psd_a_y', '0_min_psd_a_y', '0_min_max_psd_a_y', '0_max_xas_a_y', '0_min_xas_a_y', '0_min_max_xas_a_y', '0_mean_g_y', '0_var_g_y', '0_std_g_y', '0_max_g_y', '0_min_g_y', '0_range_g_y', '0_dc_comp_g_y', '0_spec_energy_g_y', '0_spec_entropy_g_y', '0_max_psd_g_y', '0_min_psd_g_y', '0_min_max_psd_g_y', '0_max_xas_g_y', '0_min_xas_g_y', '0_min_max_xas_g

In [49]:
xD.columns = col_name
xD.to_csv('sensor_data_nan.csv', index=False)

In [50]:
xD.head()

,0_mean_a_x,0_var_a_x,0_std_a_x,0_max_a_x,0_min_a_x,0_range_a_x,0_dc_comp_a_x,0_spec_energy_a_x,0_spec_entropy_a_x,0_max_psd_a_x,...,9_spec_entropy_g_m,9_max_psd_g_m,9_min_psd_g_m,9_min_max_psd_g_m,9_max_xas_g_m,9_min_xas_g_m,9_min_max_xas_g_m,activity,mat_filename,filename
0,0.08440677527151225,0.005099253183426285,0.07140905533212356,0.24560500000000002,-0.108154,0.35375900000000005,40832.2121548324,738671.4967301019,3.2067189675424275,0.4728212690903644,...,5.0868878745765835,0.1942346041985453,4.0039035887254843e-10,2.0613750084576697e-09,0.44072055114158826,2.0009756592036507e-05,4.54023678728067e-05,1,cx_a1_t1.mat,cx_a1_t1
1,0.05868655153061222,0.00557284670697336,0.07465150170608331,0.238037,-0.14868199999999998,0.386719,13230.898087460882,132037.6017507014,2.8378307843951722,0.6569401400238601,...,5.135781149742887,0.21621238847078444,5.541471171876607e-10,2.5629757901802164e-09,0.46498643901815506,2.354032958961409e-05,5.0625841130594724e-05,1,cx_a1_t10.mat,cx_a1_t10
2,0.09736474987133306,0.005283285950662429,0.07268621568538583,0.280518,-0.11377000000000001,0.39428799999999997,35788.962297324695,701134.9325655709,2.7171058541770092,0.6296151794730955,...,5.560043683051307,0.13735482472074056,2.501386569172681e-09,1.8211130000408146e-08,0.3706141183505299,5.001386376968571e-05,0.00013494861985366188,1,cx_a1_t11.mat,cx_a1_t11
3,0.06759654350927238,0.0037317322946421347,0.06108790628792359,0.17871099999999998,-0.13623,0.31494099999999997,20208.19499389196,211783.71469614343,3.3576168310313275,0.5779111521277593,...,5.161082269776515,0.11146937379258033,9.195829665884523e-11,8.249646833932981e-10,0.33387029486400904,9.589488863273434e-06,2.872219844289949e-05,1,cx_a1_t12.mat,cx_a1_t12
4,0.07944031100000015,0.005409067411984266,0.07354636233005862,0.222412,-0.09887699999999999,0.321289,25243.05204710689,374472.95690838876,2.214226934810065,0.7474342064044721,...,5.2392551699103675,0.23452612404817988,4.7834495535008886e-11,2.0396233353168795e-10,0.4842789733698748,6.916248660582476e-06,1.4281538206078782e-05,1,cx_a1_t13.mat,cx_a1_t13


,0_mean_a_x,0_mean_a_y,0_mean_a_z,0_mean_a_m,0_var_a_x,0_var_a_y,0_var_a_z,0_var_a_m,0_std_a_x,0_std_a_y,...,9_min_xas_g_y,9_min_xas_g_z,9_min_xas_g_m,9_min_max_xas_g_x,9_min_max_xas_g_y,9_min_max_xas_g_z,9_min_max_xas_g_m,mat_filename,filename,activity
0,0.08440677527151225,0.005099253183426285,0.07140905533212356,0.24560500000000002,-0.108154,0.35375900000000005,40832.2121548324,738671.4967301019,3.2067189675424275,0.4728212690903644,...,5.0868878745765835,0.1942346041985453,4.0039035887254843e-10,2.0613750084576697e-09,0.44072055114158826,2.0009756592036507e-05,4.54023678728067e-05,cx_a1_t1.mat,cx_a1_t1,1
1,0.05868655153061222,0.00557284670697336,0.07465150170608331,0.238037,-0.14868199999999998,0.386719,13230.898087460882,132037.6017507014,2.8378307843951722,0.6569401400238601,...,5.135781149742886,0.21621238847078444,5.541471171876607e-10,2.5629757901802164e-09,0.46498643901815506,2.354032958961409e-05,5.0625841130594724e-05,cx_a1_t10.mat,cx_a1_t10,1
2,0.09736474987133306,0.005283285950662429,0.07268621568538583,0.280518,-0.11377000000000001,0.39428799999999997,35788.962297324695,701134.9325655709,2.7171058541770092,0.6296151794730955,...,5.5600436830513065,0.13735482472074056,2.501386569172681e-09,1.8211130000408146e-08,0.3706141183505299,5.001386376968571e-05,0.00013494861985366188,cx_a1_t11.mat,cx_a1_t11,1
3,0.06759654350927238,0.0037317322946421347,0.06108790628792359,0.17871099999999998,-0.13623,0.31494099999999997,20208.19499389196,211783.71469614343,3.357616831031328,0.5779111521277593,...,5.161082269776515,0.11146937379258033,9.195829665884523e-11,8.249646833932981e-10,0.33387029486400904,9.589488863273434e-06,2.872219844289949e-05,cx_a1_t12.mat,cx_a1_t12,1
4,0.07944031100000015,0.005409067411984266,0.07354636233005862,0.222412,-0.09887699999999999,0.321289,25243.05204710689,374472.95690838876,2.214226934810065,0.7474342064044721,...,5.2392551699103675,0.23452612404817988,4.7834495535008886e-11,2.0396233353168795e-10,0.4842789733698748,6.916248660582476e-06,1.4281538206078782e-05,cx_a1_t13.mat,cx_a1_t13,1


In [44]:
# xD.to_csv('sensor_data_preprocessed_new.csv', index=False)